# SQL Querying

**Authors:** Ayman EL ALASS & Abderaouf KHELFAOUI  
**Course:** Databases 1 (Master 1)

## Objective
In this notebook, we perform analytical queries on our relational database `project_database.db`.
We focus on **raw data extraction and aggregation** to derive actionable business insights directly from the query results.

## Exemples
1.  **Basic Retrieval:** Extracting specific high-delay incidents.
2.  **Aggregation:** Ranking airlines by punctuality and reliability.
3.  **Complex Analysis:** Identifying the "Black List" of flight routes using double joins.
4.  **Cross-Domain Analysis:** Correlating weather conditions with flight cancellations.
5.  **Database Evolution:** Modifying the schema to permanently store delay categories.

In [33]:
import sqlite3
import pandas as pd

# Connect to the database generated by main.py
db_name = "project_database.db"
conn = sqlite3.connect(db_name)
print(f"Connected to database: {db_name}")

# Helper function to execute SQL and return a readable DataFrame
def run_query(query):
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        print(f"SQL Error: {e}")

Connected to database: project_database.db


## 1. Basic Data Retrieval (Sanity Check)
**Scenario:** A specific request from the Operations Center.
We need to list details of flights operated by 'American Airlines Inc.' that faced extreme delays (> 4 hours) to identify patterns in specific airports.

In [34]:
query_1 = """
SELECT 
    f.flight_date,
    f.flight_number,
    f.origin_airport,
    f.dest_airport,
    f.dep_delay || ' min' as delay_minutes -- Formatting for readability
FROM FLIGHTS f
JOIN AIRLINES a ON f.airline_code = a.airline_code
WHERE a.airline_name = 'American Airlines Inc.' 
  AND f.dep_delay > 240
ORDER BY f.dep_delay DESC
LIMIT 10;
"""

print(">>> Extreme Delays Report (American Airlines):")
display(run_query(query_1))

>>> Extreme Delays Report (American Airlines):


,flight_date,flight_number,origin_airport,dest_airport,delay_minutes
0,2015-01-03 00:00:00,1677,MEM,DFW,1380 min
1,2015-01-04 00:00:00,1279,OMA,DFW,1255 min
2,2015-01-01 00:00:00,2470,BOS,DFW,1190 min
3,2015-01-05 00:00:00,1495,EGE,DFW,1190 min
4,2015-01-05 00:00:00,970,LAS,LAX,1046 min
5,2015-01-03 00:00:00,2281,IND,DFW,949 min
6,2015-01-04 00:00:00,2208,CLE,DFW,925 min
7,2015-01-03 00:00:00,291,JFK,AUS,891 min
8,2015-01-03 00:00:00,45,JFK,LAS,886 min
9,2015-01-02 00:00:00,1302,RDU,DFW,885 min


## 2. Airline Performance Audit
**Exemple Question:** Which airlines are the most reliable?
We calculate three key KPIs per airline:
1.  **Volume:** Total flights.
2.  **Punctuality:** Average departure delay.
3.  **Reliability:** Cancellation Rate (%).

In [35]:
query_2 = """
SELECT 
    a.airline_name,
    COUNT(f.flight_id) as total_flights,
    ROUND(AVG(f.dep_delay), 2) as avg_delay_min,
    SUM(CASE WHEN f.cancelled = 1 THEN 1 ELSE 0 END) as cancelled_count,
    ROUND(
        (CAST(SUM(CASE WHEN f.cancelled = 1 THEN 1 ELSE 0 END) as FLOAT) / COUNT(f.flight_id)) * 100, 
    2) || '%' as cancellation_rate
FROM FLIGHTS f
JOIN AIRLINES a ON f.airline_code = a.airline_code
GROUP BY a.airline_name
HAVING total_flights > 500 -- Filter to keep only major airlines
ORDER BY avg_delay_min ASC;
"""

print(">>> Airline Performance Matrix (Ranked by Punctuality):")
display(run_query(query_2))

>>> Airline Performance Matrix (Ranked by Punctuality):


,airline_name,total_flights,avg_delay_min,cancelled_count,cancellation_rate
0,Alaska Airlines Inc.,2836,7.05,4,0.14%
1,Virgin America,1036,7.93,2,0.19%
2,Hawaiian Airlines Inc.,1328,8.52,2,0.15%
3,Delta Air Lines Inc.,13156,10.94,19,0.14%
4,US Airways Inc.,6968,11.02,62,0.89%
5,JetBlue Airways,4688,18.43,16,0.34%
6,Skywest Airlines Inc.,10475,19.93,418,3.99%
7,Southwest Airlines Co.,21160,21.39,179,0.85%
8,Atlantic Southeast Airlines,10810,21.55,421,3.89%
9,American Airlines Inc.,9449,23.24,260,2.75%


## 3. Route Analysis (Double Join)
**Context:** We want to identify the specific City-to-City connections that suffer from the worst delays.

**Technique:** We perform a **Double Join** on the `AIRPORTS` table (aliased as `origin` and `dest`) to retrieve readable city names instead of IATA codes.

In [36]:
query_3 = """
SELECT 
    origin.city || ' -> ' || dest.city AS Route,
    COUNT(*) as Flight_Count,
    ROUND(AVG(f.dep_delay), 2) as Avg_Delay_Min,
    MAX(f.dep_delay) as Max_Delay_Min
FROM FLIGHTS f
JOIN AIRPORTS origin ON f.origin_airport = origin.iata_code
JOIN AIRPORTS dest ON f.dest_airport = dest.iata_code
GROUP BY f.origin_airport, f.dest_airport
HAVING Flight_Count > 20 -- Ignore rare charter routes
ORDER BY Avg_Delay_Min DESC
LIMIT 10;
"""

print(">>> Top 10 Routes with Highest Average Delays:")
display(run_query(query_3))

>>> Top 10 Routes with Highest Average Delays:


,Route,Flight_Count,Avg_Delay_Min,Max_Delay_Min
0,San Juan -> Chicago,21,74.67,358
1,Chicago -> Mosinee,23,63.83,299
2,Columbus -> Chicago,66,59.53,739
3,Denver -> Palm Springs,27,59.04,326
4,Memphis -> Dallas-Fort Worth,33,58.06,1380
5,Lihue -> Los Angeles,28,57.75,860
6,New York -> Austin,27,57.48,891
7,Chicago -> Springfield,22,56.77,331
8,Aspen -> Chicago,25,55.96,204
9,Denver -> Colorado Springs,37,55.73,200


## 4. Exemple of Weather Impact Querying
**Context:** For instance we can assume that to avoid heavy processing times, we analyze the correlation between wind and delays for a **single specific day** (January 1st, 2015).

**Technique:** We aggregate the average wind speed and average delay per airport for that day.

In [37]:
query_4 = """
WITH DailyWeather AS (
    SELECT
        airport_code,
        AVG(wind_speed) as avg_wind_speed
    FROM WEATHER
    WHERE date(reading_time) = '2015-01-01'
    GROUP BY airport_code
),
DailyFlights AS (
    SELECT
        origin_airport,
        AVG(dep_delay) as avg_dep_delay
    FROM FLIGHTS
    WHERE date(flight_date) = '2015-01-01'
    GROUP BY origin_airport
)
SELECT
    f.origin_airport,
    f.avg_dep_delay,
    w.avg_wind_speed
FROM DailyFlights f
JOIN DailyWeather w ON f.origin_airport = w.airport_code
ORDER BY f.avg_dep_delay DESC;
"""

df_result = run_query(query_4)
display(df_result)

,origin_airport,avg_dep_delay,avg_wind_speed
0,DEN,29.416804,1.375000
1,DFW,22.316456,2.416667
2,IAH,14.187643,3.291667
3,PHX,12.929245,1.333333
4,ORD,11.043353,8.833333
5,BOS,9.355932,5.666667
6,SFO,9.248244,2.708333
7,LAX,8.199640,0.916667
8,MIA,7.330357,2.041667
9,DTW,6.750000,7.625000


## 5. Database Evolution
**Requirement:** To optimize future reporting, we need to persist the "Delay Category" directly in the database table, rather than calculating it every time.

**Actions:**
1.  **ALTER TABLE:** Add a new column `delay_category`.
2.  **UPDATE:** Populate this column based on the `dep_delay` value.

In [38]:
# 1. Add the column structure
try:
    conn.execute("ALTER TABLE FLIGHTS ADD COLUMN delay_category VARCHAR(20)")
    print("Schema Altered: Column 'delay_category' added.")
except sqlite3.OperationalError:
    print("Column 'delay_category' already exists.")

# 2. Populate the data
update_query = """
UPDATE FLIGHTS
SET delay_category = CASE
    WHEN dep_delay <= 0 THEN 'On Time / Early'
    WHEN dep_delay > 0 AND dep_delay <= 15 THEN 'Small Delay'
    WHEN dep_delay > 15 AND dep_delay <= 45 THEN 'Medium Delay'
    ELSE 'Major Delay (>45m)'
END;
"""
conn.execute(update_query)
conn.commit()
print("Data Updated: Categories populated.")

# 3. Verification Query
query_check = """
SELECT 
    delay_category, 
    COUNT(*) as flight_count,
    ROUND((CAST(COUNT(*) as FLOAT) / (SELECT COUNT(*) FROM FLIGHTS)) * 100, 1) || '%' as proportion
FROM FLIGHTS
GROUP BY delay_category
ORDER BY flight_count DESC;
"""

print(">>> Verification: Distribution of new categories:")
display(run_query(query_check))

conn.close()

Column 'delay_category' already exists.
Data Updated: Categories populated.
>>> Verification: Distribution of new categories:


,delay_category,flight_count,proportion
0,On Time / Early,45965,46.0%
1,Small Delay,22195,22.2%
2,Medium Delay,17049,17.0%
3,Major Delay (>45m),14791,14.8%
